<a href="https://colab.research.google.com/github/Jeevikavishwakarma/Credit_Card_Fraud_Detection/blob/main/Credit_card_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the dataset
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"jeevikavishwakarma","key":"d1fee7fd574a56ca920a95fc40e0d0a8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kartik2112/fraud-detection

Dataset URL: https://www.kaggle.com/datasets/kartik2112/fraud-detection
License(s): CC0-1.0
fraud-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#Extracting and loading the files
import zipfile
import pandas as pd

with zipfile.ZipFile('fraud-detection.zip') as zipref:
  zipref.extractall('fraud_detection')

df_train = pd.read_csv("/content/fraud_detection/fraudTrain.csv")
df_test = pd.read_csv("/content/fraud_detection/fraudTest.csv")

In [ ]:
df_train.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [ ]:
df_train.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [ ]:
def drop_columns(dataset):
  df = dataset.drop(['Unnamed: 0','gender','street','city','state','zip','city_pop','dob','job','category','merchant'], axis = 1)
  df[['Date','Time']] = dataset['trans_date_trans_time'].str.split(" ", n=1, expand = True)
  df['Cust_name'] = dataset[['first','last']].agg(' '.join, axis = 1)
  df = df.drop(['trans_date_trans_time','first', 'last','Date','Time','Cust_name','trans_num'], axis = 1)
  return df

df = drop_columns(df_train)
print(df)

                      cc_num     amt      lat      long   unix_time  \
0           2703186189652095    4.97  36.0788  -81.1781  1325376018   
1               630423337322  107.23  48.8878 -118.2105  1325376044   
2             38859492057661  220.11  42.1808 -112.2620  1325376051   
3           3534093764340240   45.00  46.2306 -112.1138  1325376076   
4            375534208663984   41.96  38.4207  -79.4629  1325376186   
...                      ...     ...      ...       ...         ...   
1296670       30263540414123   15.56  37.7175 -112.4777  1371816728   
1296671     6011149206456997   51.70  39.2667  -77.5101  1371816739   
1296672     3514865930894695  105.93  32.9396 -105.8189  1371816752   
1296673     2720012583106919   74.90  43.3526 -102.5411  1371816816   
1296674  4292902571056973207    4.30  45.8433 -113.8748  1371816817   

         merch_lat  merch_long  is_fraud  
0        36.011293  -82.048315         0  
1        49.159047 -118.186462         0  
2        43.150704

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

df = df.sample(frac = 0.75, random_state = 42)
#Features selection
x = df.drop('is_fraud', axis = 1)
y = df['is_fraud']

#Scaling Amount
x['amt'] = StandardScaler().fit_transform(x['amt'].values.reshape(-1,1))
#x = x.drop(['Time'], axis = 1)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2, stratify = y)
model = RandomForestClassifier(n_estimators= 80)
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


[[72440    70]
 [  291   137]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     72510
           1       0.66      0.32      0.43       428

    accuracy                           1.00     72938
   macro avg       0.83      0.66      0.71     72938
weighted avg       0.99      1.00      0.99     72938



In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9950505909128301


In [ ]:
import joblib
joblib.dump(model, 'Credit_card_fraud_model.pkl')
model = joblib.load('Credit_card_fraud_model.pkl')


In [ ]:
df1 = drop_columns(df_test)
df1 = df1.drop('is_fraud', axis = 1)
df1['prediction'] = model.predict(df1)
df1['prediction'] = df1['prediction'].map({0:'Not fraud',1:'Fraud'})
print(df1)

df1.to_csv('Credit_card_fraud_detection.csv')

                  cc_num     amt      lat      long   unix_time  merch_lat  \
0       2291163933867244    2.86  33.9659  -80.9355  1371816865  33.986391   
1       3573030041201292   29.84  40.3207 -110.4360  1371816873  39.450498   
2       3598215285024754   41.28  40.6729  -73.5365  1371816893  40.495810   
3       3591919803438423   60.05  28.5697  -80.8191  1371816915  28.812398   
4       3526826139003047    3.19  44.2529  -85.0170  1371816917  44.959148   
...                  ...     ...      ...       ...         ...        ...   
555714    30560609640617   43.77  40.4931  -91.8912  1388534347  39.946837   
555715  3556613125071656  111.84  29.0393  -95.4401  1388534349  29.661049   
555716  6011724471098086   86.88  46.1966 -118.9017  1388534355  46.658340   
555717     4079773899158    7.99  44.6255 -116.4493  1388534364  44.470525   
555718  4170689372027579   38.13  35.6665  -97.4798  1388534374  36.210097   

        merch_long prediction  
0       -81.200714  Not fraud  